<a href="https://colab.research.google.com/github/rtjord/ece570-project/blob/main/ECE_570_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cut Once, Measure Twice: A Reproduction of CutMix’s Reported Gains

>[Cut Once, Measure Twice: A Reproduction of CutMix’s Reported Gains](#scrollTo=pGH7Z0hfEAtY)

>[Runtime Setup](#scrollTo=Fl4iaIoSETEz)

>>[Set Up File Directory](#scrollTo=4zb6uHYTGawp)

>>[Print Device Info and Package Versions](#scrollTo=zV1SAaGiG1Xk)

>[Helpers](#scrollTo=IEoTzlq7wBZn)

>>[Set Seed](#scrollTo=4URjJlVfGmKh)

>>[Get Device](#scrollTo=Z2yIwlSWHrRN)

>[Load Data](#scrollTo=5dzCEj2jusP6)

>>[Make Augmentations](#scrollTo=MFQblUZxrvtO)

>>[Make Transforms](#scrollTo=wlwgTPZLIVDf)

>>[CIFAR-100 Data Loaders](#scrollTo=DPX6N4KUJIux)

>>[Download Tiny ImageNet](#scrollTo=-9t303LUJwBC)

>>[Construct Tiny ImageNet Validation](#scrollTo=wbRB1FwlKQfD)

>>[Tiny ImageNet Data Loaders](#scrollTo=WZTDt77PKdEW)

>[Build Model](#scrollTo=w6WhObE6vh5B)

>[Train/Evaluate Model](#scrollTo=6GdW2IhbvtSt)

>>[Soft Categorical Cross-Entropy](#scrollTo=06Y2AhjzO98A)

>>[Plotting Helper](#scrollTo=ub-tpV1hPVU2)

>>[Train One Epoch](#scrollTo=LfO7wYCjOW4e)

>>[Training Loop](#scrollTo=jRmkAvixWdmc)

>>[Evaluate Model](#scrollTo=m2jQXMHEvvjc)

>>[Run Experiment (Train + Test)](#scrollTo=RUA1Ycgj1hS5)

>[Global Config](#scrollTo=egwwakGrE2Dk)

>[ResNet-50 CIFAR-100](#scrollTo=119xYLjkFQhC)

>>[Baseline](#scrollTo=053-lvam1kUF)

>>[CutMix](#scrollTo=Ft9_i0T49uRm)

>>[Cutout](#scrollTo=RbCKSpVytbBK)

>>[Mixup](#scrollTo=CPCSziqMthZG)

>[ResNet-18 CIFAR-100](#scrollTo=FZjvvYroFpBh)

>>[Baseline](#scrollTo=Ddt8Gj7k8Plf)

>>[CutMix](#scrollTo=rmH6Wp23Ca1Z)

>>[Cutout](#scrollTo=e9Y80pGNkehE)

>>[MixUp](#scrollTo=_oGl1HoIkiFq)

>[ResNet-50 Tiny ImageNet](#scrollTo=hIaE-DLw4z9i)

>>[Baseline](#scrollTo=sab-6UTf5BDC)

>>[CutMix](#scrollTo=1WnYbOdHBgEs)

>[Create Figures](#scrollTo=gH90e69ybkUF)

>>[Training History](#scrollTo=Lgk24c783Q7A)

>>[Data Augmentation Examples](#scrollTo=TgIT35UtnnWl)



# Runtime Setup

## Set Up File Directory

In [ ]:
import os, random, platform
from pathlib import Path
import numpy as np
import torch
import torchvision
import matplotlib

# ---- Base paths ----
BASE_DIR = Path("./")
DATA_DIR = BASE_DIR / "data"
CKPT_DIR = BASE_DIR / "checkpoints"
RESULTS_DIR = BASE_DIR / "results"
FIG_DIR = BASE_DIR / "figures"
for d in [DATA_DIR, CKPT_DIR, RESULTS_DIR, FIG_DIR]:
    d.mkdir(parents=True, exist_ok=True)


## Print Device Info and Package Versions

In [ ]:
import torch
import torchvision
import matplotlib
import platform

# ---- Device info ----
if torch.cuda.is_available():
    device = torch.device("cuda")
    props = torch.cuda.get_device_properties(0)
    cc = f"{props.major}.{props.minor}"
    print(f"Device: CUDA — {torch.cuda.get_device_name(0)} (cc {cc})")
    print(f"GPU RAM: {props.total_memory/1e9:.2f} GB")
else:
    device = torch.device("cpu")
    print("Device: CPU")

# ---- Versions ----
print(f"PyTorch:     {torch.__version__} (CUDA {torch.version.cuda})")
print(f"Torchvision: {torchvision.__version__}")
print(f"Matplotlib:  {matplotlib.__version__}")
print(f"Python:      {platform.python_version()}")

# ---- Paths summary ----
print(f"DATA_DIR = {DATA_DIR}")
print(f"CKPT_DIR = {CKPT_DIR}")
print(f"FIG_DIR  = {FIG_DIR}")

# Helpers

## Set Seed



In [ ]:
import random
import numpy as np
import torch


def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True

## Get Device

In [ ]:
import torch

def get_device():
    """Prefer CUDA, then Apple MPS, else CPU."""
    if torch.cuda.is_available():
        return torch.device("cuda")
    if getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
        return torch.device("mps")
    return torch.device("cpu")

# Load Data

## Make Augmentations

In [ ]:
from torch import nn
from torchvision.transforms import v2 as T2


def make_batch_aug(num_classes: int, mode: str = "none", p: float = 0.5):
    """
    Construct a batch-level augmentation module (MixUp, CutMix, Cutout)
    or return None if mode is 'none'/'off'.
    """
    m = (mode or "none").lower()
    if m in ("none", "off"):
        return None

    if m == "mixup":
        return T2.RandomApply(
            nn.ModuleList([T2.MixUp(num_classes=num_classes, alpha=1.0)]),
            p=p,
        )

    if m == "cutmix":
        return T2.RandomApply(
            nn.ModuleList([T2.CutMix(num_classes=num_classes, alpha=1.0)]),
            p=p,
        )

    if m == "cutout":
        # Use RandomErasing to mimic Cutout-style behavior
        return T2.RandomErasing(
            p=p,                 # apply with probability p
            scale=(0.25, 0.25),  # fixed area ~25% of the image
            ratio=(1.0, 1.0),
            value=0.0,
            inplace=True,
        )

    raise ValueError("mix mode must be one of: 'none', 'mixup', 'cutmix', 'cutout'")

## Make Transforms

In [ ]:
from torchvision import transforms, models

CIFAR100_MEAN = (0.5071, 0.4867, 0.4408)
CIFAR100_STD  = (0.2675, 0.2565, 0.2761)

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD  = (0.229, 0.224, 0.225)

def make_cifar_transforms(train: bool = True, architecture="resnet18"):
    if train:
        return transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(CIFAR100_MEAN, CIFAR100_STD),
        ])
    else:
        return transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(CIFAR100_MEAN, CIFAR100_STD),
        ])

def make_imagenet_transforms(train: bool = True, architecture="resnet18"):
    if train:
        return transforms.Compose([
            transforms.RandomResizedCrop(64),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
        ])
    else:
        weights = models.ResNet50_Weights.IMAGENET1K_V1
        return weights.transforms()

## CIFAR-100 Data Loaders

In [ ]:
from pathlib import Path
import torch
from torch.utils.data import DataLoader, random_split, Dataset, Subset
from torchvision import datasets
from typing import Tuple, List


def build_cifar100_loaders(
    architecture: str,
    batch_size: int,
    num_workers: int,
    seed: int,
    root: str | Path = "./data",
) -> Tuple[DataLoader, DataLoader, DataLoader, List[str]]:

    root = Path(root)
    pin  = torch.cuda.is_available()

    # Per-sample transforms
    tf_train = make_cifar_transforms(train=True, architecture=architecture)   # augments + ImageNet norm
    tf_eval  = make_cifar_transforms(train=False, architecture=architecture)  # built-in eval transform

    # Two logically identical train datasets, differing only by transform
    train_ds_aug  = datasets.CIFAR100(root=str(root), train=True,  download=True,  transform=tf_train)
    train_ds_eval = datasets.CIFAR100(root=str(root), train=True,  download=False, transform=tf_eval)
    test_ds       = datasets.CIFAR100(root=str(root), train=False, download=True,  transform=tf_eval)

    class_names = list(train_ds_aug.classes)

    # Deterministic split on indices (shared across both train datasets)
    n_total = len(train_ds_aug)
    n_val   = int(round(n_total * 0.1))
    n_train = n_total - n_val

    # random_split returns Subset objects; we only need their indices
    g = torch.Generator().manual_seed(seed)  # isolate the randomness of the split
    train_subset_idx, val_subset_idx = random_split(range(n_total), [n_train, n_val], generator=g)
    train_idx = list(train_subset_idx.indices)
    val_idx   = list(val_subset_idx.indices)

    # Subset with appropriate transforms
    train_subset = Subset(train_ds_aug,  train_idx)  # aug pipeline
    val_subset   = Subset(train_ds_eval, val_idx)    # eval pipeline (SAME as test)

    # DataLoaders
    train_loader = DataLoader(
        train_subset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=pin
    )
    val_loader = DataLoader(
        val_subset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=pin
    )
    test_loader  = DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=pin
    )

    return train_loader, val_loader, test_loader, class_names

## Download Tiny ImageNet

In [ ]:
from pathlib import Path
import subprocess

def ensure_tinyimagenet(root: str | Path = "./data") -> Path:
    """
    Ensure Tiny ImageNet is present under root.
    Downloads and unzips into root/tiny-imagenet-200 if missing.
    """
    root = Path(root)
    target_dir = root / "tiny-imagenet-200"
    zip_path = root / "tiny-imagenet-200.zip"

    if not target_dir.exists():
        root.mkdir(parents=True, exist_ok=True)

        # Download if zip not already there
        if not zip_path.exists():
            print("Downloading Tiny ImageNet...")
            subprocess.run(
                ["wget", "http://cs231n.stanford.edu/tiny-imagenet-200.zip", "-O", str(zip_path)],
                check=True
            )

        # Unzip into root
        print("Unzipping Tiny ImageNet...")
        print(zip_path)
        subprocess.run(["unzip", str(zip_path), "-d", str(root)], check=True)

    return target_dir

## Construct Tiny ImageNet Validation

In [ ]:
from pathlib import Path
import os
import subprocess
from typing import Optional
import torch
from torch.utils.data import DataLoader, random_split, Dataset, Subset
from torchvision import transforms, models
from PIL import Image


# --- Tiny ImageNet helpers ---

class TinyImageNetVal(Dataset):
    """
    Tiny-ImageNet 'val' folder uses a CSV-style annotation file instead of class subfolders.
    This dataset reads 'val_annotations.txt' and maps each image to the corresponding wnid.
    Expects the standard layout:
      root/
        train/<wnid>/*.JPEG
        val/images/*.JPEG
        val/val_annotations.txt  (filename, wnid, x, y, w, h ...)
        wnids.txt (optional)
    """
    def __init__(self, root: str | Path, transform=None, class_to_idx: Optional[dict] = None):
        self.root = Path(root)
        self.transform = transform
        anno_path = self.root / "val" / "val_annotations.txt"
        images_dir = self.root / "val" / "images"

        # Build class_to_idx from train if not given
        if class_to_idx is None:
            train_dir = self.root / "train"
            wnids = sorted([d.name for d in train_dir.iterdir() if d.is_dir()])
            class_to_idx = {wnid: i for i, wnid in enumerate(wnids)}
        self.class_to_idx = class_to_idx

        # Parse annotations
        self.samples = []  # list of (image_path, target_idx)
        with open(anno_path, "r") as f:
            for line in f:
                parts = line.strip().split("\t")
                if len(parts) < 2:
                    parts = line.strip().split()  # fallback on whitespace
                fname, wnid = parts[0], parts[1]
                if wnid not in self.class_to_idx:
                    # If class not in mapping (shouldn't happen), skip
                    continue
                img_path = images_dir / fname
                self.samples.append((img_path, self.class_to_idx[wnid]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, target = self.samples[idx]
        img = Image.open(path).convert("RGB")
        if self.transform is not None:
            img = self.transform(img)
        return img, target

## Tiny ImageNet Data Loaders

In [ ]:
from pathlib import Path
from torch.utils.data import DataLoader, random_split
from typing import Tuple, List
import torch
from torch.utils.data import DataLoader, random_split, Subset
from torchvision import datasets


def build_tinyimagenet_loaders(
    architecture: str,
    batch_size: int,
    num_workers: int,
    seed: int,
    root: str | Path = "./data",
) -> Tuple[DataLoader, DataLoader, DataLoader, List[str]]:
    """
    Returns train_loader, val_loader, test_loader, class_names for Tiny ImageNet.
    - train: ImageFolder(root/train)
    - val: uses TinyImageNetVal (annotation file) OR ImageFolder if already reorganized
    - test: labels are not provided in the official release; we reuse 'val' as 'test' for metrics.
    """
    root = Path(root)
    ensure_tinyimagenet(root)
    pin = torch.cuda.is_available()

    tf_train = make_imagenet_transforms(train=True, architecture=architecture)
    tf_eval  = make_imagenet_transforms(train=False, architecture=architecture)

    train_dir = root / "tiny-imagenet-200" / "train"

    # Two logically identical train datasets, differing only by transform
    train_ds_aug  = datasets.ImageFolder(str(train_dir), transform=tf_train)
    train_ds_eval = datasets.ImageFolder(str(train_dir), transform=tf_eval)
    assert len(train_ds_aug.classes) == 200, f"Expected 200 train classes, got {len(train_ds_aug.classes)}"
    class_names = list(train_ds_aug.classes)  # wnids

    # Deterministic split on indices (shared across both train datasets)
    n_total = len(train_ds_aug)
    n_val   = int(round(n_total * 0.1))
    n_train = n_total - n_val

    # random_split returns Subset objects; we only need their indices
    g = torch.Generator().manual_seed(seed)  # isolate the randomness of the split
    train_subset_idx, val_subset_idx = random_split(range(n_total), [n_train, n_val], generator=g)
    train_idx = list(train_subset_idx.indices)
    val_idx   = list(val_subset_idx.indices)

    # Subset with appropriate transforms
    train_subset = Subset(train_ds_aug,  train_idx)  # aug pipeline
    val_subset   = Subset(train_ds_eval, val_idx)    # eval pipeline (SAME as test)

    # Use validation set as test set
    test_set = TinyImageNetVal(
        root / "tiny-imagenet-200",
        transform=tf_eval,
        class_to_idx=train_ds_aug.class_to_idx
    )

    # DataLoaders
    train_loader = DataLoader(
        train_subset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=pin,
        persistent_workers=(num_workers > 0),
        prefetch_factor=2 if num_workers > 0 else None,
    )
    val_loader = DataLoader(
        val_subset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=pin,
        persistent_workers=(num_workers > 0),
        prefetch_factor=2 if num_workers > 0 else None,
    )
    test_loader  = DataLoader(
        test_set,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=pin,
        persistent_workers=(num_workers > 0),
        prefetch_factor=2 if num_workers > 0 else None,
    )

    return train_loader, val_loader, test_loader, class_names

# Build Model

In [ ]:
import torchvision
from torch import nn

import torchvision
from torch import nn

def build_cifar100_model(architecture: str):
    """
    CIFAR-style ResNet stem: 3x3 conv, stride 1, no maxpool.
    Good for 32x32 inputs when training from scratch.
    """
    if architecture == "resnet18":
        model = torchvision.models.resnet18(weights=None)
    elif architecture == "resnet50":
        model = torchvision.models.resnet50(weights=None)
    else:
        raise ValueError(f"Unknown architecture {architecture}")

    model.conv1 = nn.Conv2d(
        3, 64, kernel_size=3, stride=1, padding=1, bias=False
    )
    model.maxpool = nn.Identity()

    model.fc = nn.Linear(model.fc.in_features, 100)
    return model


def build_tiny_imagenet_model(architecture: str):
    """
    Original ResNet model with fully connected layer swapped for
    correct number of classes
    """
    if architecture == "resnet18":
        model = torchvision.models.resnet18(weights=None)
    elif architecture == "resnet50":
        model = torchvision.models.resnet50(weights=None)
    else:
        raise ValueError(f"Unknown architecture {architecture}")

    model.conv1 = nn.Conv2d(
        3, 64, kernel_size=3, stride=1, padding=1, bias=False
    )
    # model.maxpool = nn.Identity()

    model.fc = nn.Linear(model.fc.in_features, 200)
    return model

# Train/Evaluate Model

## Soft Categorical Cross-Entropy

In [ ]:
from torch import nn

class SoftCE(nn.Module):
    def forward(self, logits, targets):
        if targets.dtype in (torch.long, torch.int64):  # hard labels
            return nn.functional.cross_entropy(logits, targets)
        logp = nn.functional.log_softmax(logits, dim=-1)
        return -(targets * logp).sum(dim=-1).mean()

## Plotting Helper

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_curves(history: dict, title: str):
    """Plot train loss and val top-1 from a History dict."""
    if not history or "train_loss" not in history:
        print("Nothing to plot.")
        return
    xs = np.arange(1, len(history["train_loss"]) + 1)
    fig, ax1 = plt.subplots()
    ax1.plot(xs, history["train_loss"], label="train_loss")
    ax1.plot(xs, history.get("val_loss", []), label="val_loss")
    ax1.set_xlabel("epoch"); ax1.set_ylabel("loss"); ax1.grid(True, alpha=0.3)
    ax2 = ax1.twinx()
    if "val_top1" in history:
        ax2.plot(xs, history["val_top1"], "o--", label="val_top1", alpha=0.7)
        ax2.set_ylabel("Top-1 (%)")
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines + lines2, labels + labels2, loc="best")
    plt.title(title)
    plt.savefig(FIG_DIR / f"{title}.png")
    plt.show()

## Train One Epoch

In [ ]:
import torch
from torch import nn


def train_one_epoch(
    model,
    loader,
    optimizer,
    *,
    device=None,
    batch_aug=None,
    amp=True,
):
    """One training epoch. Returns average loss."""
    device = device or get_device()
    model.train()
    criterion = SoftCE()

    scaler = torch.amp.GradScaler(
        device="cuda",
        enabled=(amp and device.type == "cuda")
    )
    running_loss, seen = 0.0, 0

    for x, y in loader:
        x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)

        if batch_aug is not None:
            x, y = batch_aug(x, y)  # may return soft targets

        optimizer.zero_grad(set_to_none=True)
        with torch.autocast(device_type=device.type, enabled=(amp and device.type in {"cuda", "mps"})):
            logits = model(x)
            loss = criterion(logits, y)

        if scaler.is_enabled():
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()

        bs = y.size(0)
        running_loss += loss.item() * bs
        seen += bs

    return running_loss / max(1, seen)

## Training Loop

In [ ]:
import numpy as np
import copy

def cpu_state_dict(model):
    return {k: v.detach().cpu() for k, v in model.state_dict().items()}

def train(
    model,
    train_loader,
    val_loader,
    *,
    device,
    epochs,
    batch_aug=None,
    optimizer=None,
    scheduler=None,
    amp=True,
    save_path=None
):
    model.to(device)

    history = {
        "train_loss": [],
        "val_loss": [],
        "val_top1": [],
        "val_top5": [],
        "lr": []
    }

    best_val_loss = np.inf
    best_model_checkpoint = None

    for epoch in range(1, epochs + 1):
        train_loss = train_one_epoch(
            model,
            train_loader,
            optimizer,
            device=device,
            batch_aug=batch_aug,
            amp=amp
        )

        val_metrics = evaluate(model, val_loader, device=device)
        val_loss = val_metrics["loss"]
        top1 = val_metrics["top1"]
        top5 = val_metrics["top5"]

        if scheduler is not None:
          # get the learning rate before stepping the scheduler
          lr = scheduler.get_last_lr()[0]
          scheduler.step()
        else:
          lr = optimizer.param_groups[0]["lr"]

        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)
        history["val_top1"].append(top1)
        history["val_top5"].append(top5)
        history["lr"].append(lr)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_checkpoint = {
                  "epoch": epoch,
                  "model_state": cpu_state_dict(model),
            }


        print(
            f"epoch {epoch:03d} "
            f"train_loss={train_loss:.4f}  val_loss={val_loss:.4f}  "
            f"top1={top1:.2f}%  top5={top5:.2f}%  "
            f"lr={lr}"
        )

    # Save at the end once the entire training history has finished
    if best_model_checkpoint is not None:
      best_model_checkpoint["history"] = history
      if save_path is not None:
          torch.save(best_model_checkpoint, save_path)

    return best_model_checkpoint


## Evaluate Model

In [ ]:
def evaluate(model, loader, device=None, criterion=None, max_batches=None):
    """Eval with hard labels (no batch-level aug). Returns dict: loss/top1/top5."""
    device = device or get_device()
    model.eval()
    criterion = criterion or nn.CrossEntropyLoss()

    total, loss_sum, top1_sum, top5_sum = 0, 0.0, 0.0, 0.0
    with torch.no_grad():
        for b, (x, y) in enumerate(loader):
            x, y = x.to(device), y.to(device)
            logits = model(x)
            loss_sum += criterion(logits, y).item() * y.size(0)
            total += y.size(0)

            # Top-1 / Top-5
            pred1 = logits.argmax(dim=1)
            top1_sum += (pred1 == y).sum().item()
            k = min(5, logits.size(1))
            pred5 = logits.topk(k, dim=1).indices
            top5_sum += (pred5 == y.view(-1, 1)).any(dim=1).sum().item()

            # Limit number of batches used for evaluation
            if max_batches is not None and (b + 1) >= max_batches:
                break

    return {
        "loss": loss_sum / max(1, total),
        "top1": 100.0 * top1_sum / max(1, total),
        "top5": 100.0 * top5_sum / max(1, total),
    }

## Run Experiment (Train + Test)

In [ ]:
import time
import torch
from torch import nn
import numpy as np
from torchvision.transforms import v2 as T2
from google.colab import files

def run_experiment(name: str, cfg: dict):
    """
    Returns:
      {
        'name': ...,
        'history': {...},
        'test_metrics': {'loss': ..., 'top1': ..., 'top5': ...},
      }
    """
    set_seed(cfg["seed"])
    device = get_device()

    # build data loaders
    if cfg["dataset"] == "cifar100":
        train_loader, val_loader, test_loader, class_names = build_cifar100_loaders(
            architecture=cfg["architecture"],
            batch_size=cfg["batch_size"],
            num_workers=cfg["num_workers"],
            seed=cfg["seed"],
        )
        model = build_cifar100_model(
            architecture=cfg["architecture"],
        ).to(device)
        num_classes = 100
    elif cfg["dataset"] == "tinyimagenet":
        train_loader, val_loader, test_loader, class_names = build_tinyimagenet_loaders(
            architecture=cfg["architecture"],
            batch_size=cfg["batch_size"],
            num_workers=cfg["num_workers"],
            seed=cfg["seed"],
        )
        model = build_tiny_imagenet_model(
            architecture=cfg["architecture"],
        ).to(device)
        num_classes = 200
    else:
        raise Exception(f"Unknown dataset {cfg["dataset"]}")

    print("\n--- Dataset split ---")
    sizes = {
        "train": len(train_loader.dataset),
        "val":   len(val_loader.dataset),
        "test":  len(test_loader.dataset),
    }
    print(sizes)

    # These settings are the same whether fine tuning or training from scratch
    batch_aug = make_batch_aug(num_classes, cfg["aug_mode"], p=cfg["aug_prob"])

    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(
        params,
        lr=cfg["lr"],
        momentum=0.9,
        nesterov=True,
        weight_decay=cfg["weight_decay"],
    )

    scheduler = torch.optim.lr_scheduler.MultiStepLR(
        optimizer,
        milestones=[int(0.25*cfg["epochs"]),
                    int(0.50*cfg["epochs"]),
                    int(0.75*cfg["epochs"])],
        gamma=0.1
    )

    print("\n--- Train ---")
    best_model_checkpoint = train(
        model,
        train_loader,
        val_loader,
        device=device,
        epochs=cfg["epochs"],
        batch_aug=batch_aug,
        optimizer=optimizer,
        scheduler=scheduler,
        amp=cfg["amp"],
        save_path=CKPT_DIR / f"{name}.pt"
    )
    hist = best_model_checkpoint["history"]
    plot_curves(hist, title=name)


    print("\n--- Test ---")
    # Load weights from the epoch with lowest validation loss
    model.load_state_dict(best_model_checkpoint["model_state"])

    test_metrics = evaluate(model, test_loader, device=device)
    print(
        f"loss={test_metrics['loss']:.4f}  "
        f"top1={test_metrics['top1']:.2f}%  "
        f"top5={test_metrics['top5']:.2f}%"
    )

    results = best_model_checkpoint | {
        "name": name,
        "test_metrics": test_metrics,
    }

    torch.save(results, RESULTS_DIR / f"{name}.pt")
    files.download(RESULTS_DIR / f"{name}.pt")
    return results


# Global Config
Keep these hyperparameters the same between experiments.

In [ ]:
GLOBAL_CONFIG = {
    "seed": 42,
    "batch_size": 256,
    "num_workers": 4,
    "aug_prob": 0.5,
    "lr": 0.1,
    "weight_decay": 5e-4,
    "amp": True,
}

# ResNet-50 CIFAR-100

## Baseline

In [ ]:
CONFIG = GLOBAL_CONFIG | {
    "dataset": "cifar100",
    "aug_mode": None,
    "architecture": "resnet50",
    "epochs": 300,
}

resnet50_cifar_baseline = run_experiment("resnet50_cifar_baseline", CONFIG)

## CutMix

In [ ]:
CONFIG = GLOBAL_CONFIG | {
    "dataset": "cifar100",
    "aug_mode": "cutmix",
    "architecture": "resnet50",
    "epochs": 300,
}

resnet50_cifar_cutmix = run_experiment("resnet50_cifar_cutmix", CONFIG)

## Cutout

In [ ]:
CONFIG = GLOBAL_CONFIG | {
    "dataset": "cifar100",
    "aug_mode": "cutout",
    "architecture": "resnet50",
    "epochs": 300,
}

resnet50_cifar_cutout = run_experiment("resnet50_cifar_cutout", CONFIG)

## Mixup

In [ ]:
CONFIG = GLOBAL_CONFIG | {
    "dataset": "cifar100",
    "aug_mode": "mixup",
    "architecture": "resnet50",
    "epochs": 300,
}

resnet50_cifar_mixup = run_experiment("resnet50_cifar_mixup", CONFIG)

# ResNet-18 CIFAR-100

## Baseline

In [ ]:
CONFIG = GLOBAL_CONFIG | {
    "dataset": "cifar100",
    "aug_mode": None,
    "architecture": "resnet18",
    "epochs": 200,
}

resnet18_cifar_baseline = run_experiment("resnet18_cifar_baseline", CONFIG)

## CutMix

In [ ]:
CONFIG = GLOBAL_CONFIG | {
    "dataset": "cifar100",
    "aug_mode": "cutmix",
    "architecture": "resnet18",
    "epochs": 200,
}

resnet18_cifar_cutmix = run_experiment("resnet18_cifar_cutmix", CONFIG)

## Cutout

In [ ]:
CONFIG = GLOBAL_CONFIG | {
    "dataset": "cifar100",
    "aug_mode": "cutout",
    "architecture": "resnet18",
    "epochs": 200,
}

resnet18_cifar_cutout = run_experiment("resnet18_cifar_cutout", CONFIG)

## MixUp

In [ ]:
CONFIG = GLOBAL_CONFIG | {
    "dataset": "cifar100",
    "aug_mode": "mixup",
    "architecture": "resnet18",
    "epochs": 200,
}

resnet18_cifar_mixup = run_experiment("resnet18_cifar_mixup", CONFIG)

# ResNet-50 Tiny ImageNet

## Baseline

In [ ]:
CONFIG = GLOBAL_CONFIG | {
    "dataset": "tinyimagenet",
    "aug_mode": None,
    "architecture": "resnet50",
    "epochs": 300,
}

resnet50_tiny_baseline = run_experiment("resnet50_tiny_baseline", CONFIG)

## CutMix

In [ ]:
CONFIG = GLOBAL_CONFIG | {
    "dataset": "tinyimagenet",
    "aug_mode": "cutmix",
    "architecture": "resnet50",
    "epochs": 300,
}

resnet50_tiny_cutmix = run_experiment("resnet50_tiny_cutmix", CONFIG)

# Create Figures

## Training History

In [ ]:
from pathlib import Path
from typing import Dict, List, Sequence, Tuple, Optional, Callable

import torch
import matplotlib.pyplot as plt

# -----------------------------
# Loading utilities
# -----------------------------

def load_runs(run_specs: Sequence[Tuple[str, str]], base_dir: Path) -> List[Dict]:
    """
    run_specs: list of (relative_path, label)
      relative_path is relative to base_dir (no suffix assumptions).
      label is what will appear in the legend.
    Returns a list of dicts with fields: label, history.
    """
    runs = []
    for rel_path, label in run_specs:
        ckpt_path = base_dir / rel_path
        ckpt = torch.load(ckpt_path, map_location="cpu")

        history = ckpt.get("history", ckpt)  # supports either layout
        runs.append({"label": label, "history": history})
    return runs


# -----------------------------
# Plotting utilities
# -----------------------------

def plot_metric(
    runs: Sequence[Dict],
    metric_key: str,
    *,
    title: str,
    ylabel: str,
    xlabel: str = "Epoch",
    out_dir: Path = Path("figures"),
    out_name: Optional[str] = None,
    dpi: int = 300,
    x_lim: Optional[Tuple[float, float]] = None,
    y_lim: Optional[Tuple[float, float]] = None,
    show: bool = True,
    transform: Optional[Callable[[List[float]], List[float]]] = None,
):
    """
    metric_key should be something like:
      'train_loss', 'val_loss', 'val_top1', 'val_top5', 'lr', etc.
    transform: optional function applied to the series before plotting
               (e.g., accuracy -> error).
    Saves both PNG and PDF for LaTeX.
    """
    out_dir.mkdir(parents=True, exist_ok=True)
    out_name = out_name or metric_key

    plt.figure(figsize=(6, 5))

    for r in runs:
        series = r["history"][metric_key]
        if transform is not None:
            series = transform(series)
        plt.plot(series, label=r["label"])

    plt.title(title, fontsize=14)
    plt.xlabel(xlabel, fontsize=14)
    plt.ylabel(ylabel, fontsize=14)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.legend(fontsize=14)
    plt.grid(True, alpha=0.3)
    if x_lim is not None:
        plt.xlim(*x_lim)
    if y_lim is not None:
        plt.ylim(*y_lim)
    plt.tight_layout()

    png_path = out_dir / f"{out_name}.png"
    pdf_path = out_dir / f"{out_name}.pdf"
    plt.savefig(png_path, dpi=dpi, bbox_inches="tight")
    plt.savefig(pdf_path, bbox_inches="tight")  # vector, great for LaTeX

    if show:
        plt.show()
    else:
        plt.close()


acc_to_err = lambda s: [100.0 - v for v in s]


# -----------------------------
# Example usage
# -----------------------------


resnet50_cifar_specs = [
    (r"resnet50_cifar_baseline.pt", "Baseline"),
    # (r"resnet50_cifar_cutout.pt",   "Cutout"),
    # (r"resnet50_cifar_mixup.pt",    "MixUp"),
    (r"resnet50_cifar_cutmix.pt",   "CutMix"),
]
resnet18_cifar_specs = [
    (r"resnet18_cifar_baseline.pt", "Baseline"),
    (r"resnet18_cifar_cutout.pt",   "Cutout"),
    (r"resnet18_cifar_mixup.pt",    "MixUp"),
    (r"resnet18_cifar_cutmix.pt",   "CutMix"),
]
resnet50_tiny_specs = [
    (r"resnet50_tiny_baseline.pt", "Baseline"),
    (r"resnet50_tiny_cutmix.pt",   "CutMix"),
]

runs_50_cifar = load_runs(resnet50_cifar_specs, RESULTS_DIR)
runs_18_cifar = load_runs(resnet18_cifar_specs, RESULTS_DIR)
runs_50_tiny = load_runs(resnet50_tiny_specs, RESULTS_DIR)


plot_metric(
    runs_50_cifar, "val_top1",
    title=f"ResNet-50 CIFAR-100: Validation Top-1 Error",
    ylabel="Top-1 Error (%)",
    out_dir=Path("figures/cifar100"),
    out_name=f"ResNet-50 CIFAR-100_val_top1_err",
    x_lim=(0, 300),
    y_lim=(15, 100),
    dpi=400,
    show=True,
    transform=acc_to_err,
)

plot_metric(
    runs_18_cifar, "val_top1",
    title=f"ResNet-18 CIFAR-100: Validation Top-1 Error",
    ylabel="Top-1 Error (%)",
    out_dir=Path("figures/cifar100"),
    out_name=f"ResNet-18 CIFAR-100_val_top1_err",
    x_lim=(0, 200),
    y_lim=(15, 100),
    dpi=400,
    show=True,
    transform=acc_to_err,
)

plot_metric(
    runs_50_tiny, "val_top1",
    title=f"ResNet-50 Tiny ImageNet: Validation Top-1 Error",
    ylabel="Top-1 Error (%)",
    out_dir=Path("figures/tinyimagenet"),
    out_name=f"ResNet-50 Tiny ImageNet_val_top1_err",
    x_lim=(0, 300),
    y_lim=(15, 100),
    dpi=400,
    show=True,
    transform=acc_to_err,
)


## Data Augmentation Examples

In [ ]:
# --- Colab setup ---
import math
import random
from pathlib import Path

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torchvision.transforms.functional as TF


# -------------------------
# Utilities
# -------------------------

def load_image(path, size=None):
    """Load RGB image. Optionally resize to (H,W)."""
    img = Image.open(path).convert("RGB")
    if size is not None:
        img = img.resize((size[1], size[0]), Image.BILINEAR)  # PIL takes (W,H)
    return img

def pil_to_tensor(img):
    """PIL -> float tensor in [0,1], shape (C,H,W)."""
    return TF.to_tensor(img)

def tensor_to_pil(x):
    """float tensor (C,H,W) in [0,1] -> PIL."""
    x = x.clamp(0, 1)
    return TF.to_pil_image(x)

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)


# -------------------------
# Augmentations (deterministic / controllable)
# -------------------------

def apply_mixup(x1, x2, lam=0.5):
    """
    x1, x2: float tensors (C,H,W) in [0,1]
    lam: mixing coefficient for primary image
    """
    return lam * x1 + (1 - lam) * x2

def apply_cutout(x, erase_frac=0.25, fill=0.0, center=None, seed=None):
    """
    x: float tensor (C,H,W)
    erase_frac: fraction of total area to erase (e.g., 0.25 = 25%)
    fill: fill value in [0,1]
    center: (cy, cx) in pixels. If None, random location.
    seed: for reproducible random location.
    """
    if seed is not None:
        set_seed(seed)

    C, H, W = x.shape
    erase_area = erase_frac * H * W
    side = int(round(math.sqrt(erase_area)))  # square patch

    if center is None:
        cy = random.randint(0, H-1)
        cx = random.randint(0, W-1)
    else:
        cy, cx = center

    y1 = max(0, cy - side // 2)
    y2 = min(H, y1 + side)
    x1 = max(0, cx - side // 2)
    x2 = min(W, x1 + side)

    out = x.clone()
    out[:, y1:y2, x1:x2] = fill
    return out

def rand_bbox(H, W, lam, seed=None):
    """Standard CutMix bbox sampling given lambda."""
    if seed is not None:
        set_seed(seed)

    cut_ratio = math.sqrt(1.0 - lam)
    cut_w = int(W * cut_ratio)
    cut_h = int(H * cut_ratio)

    cx = random.randint(0, W-1)
    cy = random.randint(0, H-1)

    x1 = np.clip(cx - cut_w // 2, 0, W)
    x2 = np.clip(cx + cut_w // 2, 0, W)
    y1 = np.clip(cy - cut_h // 2, 0, H)
    y2 = np.clip(cy + cut_h // 2, 0, H)

    return int(x1), int(y1), int(x2), int(y2)

def apply_cutmix(x1, x2, lam=0.5, bbox=None, seed=None):
    """
    x1 primary, x2 secondary. Pastes patch from x2 into x1.
    lam: area fraction kept from primary (approx; exact depends on bbox)
    bbox: (x1,y1,x2,y2). If None, sample from lam.
    seed: reproducible random bbox sampling.
    Returns: cutmixed tensor, effective_lambda
    """
    C, H, W = x1.shape
    if bbox is None:
        x1b, y1b, x2b, y2b = rand_bbox(H, W, lam, seed=seed)
    else:
        x1b, y1b, x2b, y2b = bbox

    out = x1.clone()
    out[:, y1b:y2b, x1b:x2b] = x2[:, y1b:y2b, x1b:x2b]

    # effective lambda based on actual area
    patch_area = (x2b - x1b) * (y2b - y1b)
    eff_lam = 1.0 - patch_area / (H * W)
    return out, eff_lam


# -------------------------
# Figure creation
# -------------------------

def make_augmentation_row(
    primary_path,
    secondary_path,
    *,
    primary_label="Primary",
    secondary_label="Secondary",
    out_path="augmentation_row.png",
    size=(64, 64),  # (H,W); set None to keep original
    mixup_lam=0.5,
    cutout_frac=0.25,
    cutout_fill=0.0,
    cutout_center=None,  # e.g., (H//2, W//2)
    cutmix_lam=0.5,
    cutmix_bbox=None,    # e.g., (x1,y1,x2,y2)
    seed=0,
    dpi=400,
):
    """
    Produces 1x4 row: Baseline | MixUp | Cutout | CutMix.
    Labels technique above, class/mix label below.
    """
    set_seed(seed)

    # Load + match sizes
    img1 = load_image(primary_path, size=size)
    img2 = load_image(secondary_path, size=size)

    x1 = pil_to_tensor(img1)
    x2 = pil_to_tensor(img2)

    # Baseline
    baseline_img = tensor_to_pil(x1)
    baseline_text = primary_label + " 1.0"

    # MixUp
    mixup_x = apply_mixup(x1, x2, lam=mixup_lam)
    mixup_img = tensor_to_pil(mixup_x)
    mixup_text = f"{primary_label} {mixup_lam:.2f} \n {secondary_label} {1-mixup_lam:.2f}"

    # Cutout
    cutout_x = apply_cutout(
        x1, erase_frac=cutout_frac, fill=cutout_fill,
        center=cutout_center, seed=seed
    )
    cutout_img = tensor_to_pil(cutout_x)
    cutout_text = primary_label + " 1.0"

    # CutMix
    cutmix_x, eff_lam = apply_cutmix(
        x1, x2, lam=cutmix_lam,
        bbox=cutmix_bbox, seed=seed
    )
    cutmix_img = tensor_to_pil(cutmix_x)
    cutmix_text = f"{primary_label} {eff_lam:.2f} \n {secondary_label} {1-eff_lam:.2f}"

    panels = [
        ("Baseline", baseline_img, baseline_text),
        ("Mixup",    mixup_img,    mixup_text),
        ("Cutout",   cutout_img,   cutout_text),
        ("CutMix",   cutmix_img,   cutmix_text),
    ]

    fig, axes = plt.subplots(1, 4, figsize=(14, 4))
    for ax, (title, im, lbl) in zip(axes, panels):
        ax.imshow(im)
        ax.set_title(title, fontsize=20, pad=10)   # technique ABOVE
        ax.set_xlabel(lbl, fontsize=20, labelpad=8)  # label BELOW
        ax.set_xticks([])
        ax.set_yticks([])

    plt.tight_layout()
    plt.savefig(out_path, dpi=dpi, bbox_inches="tight")
    plt.show()
    print(f"Saved to {out_path}")


# -------------------------
# Example usage (edit these)
# -------------------------

ensure_tinyimagenet()
# Put your two image paths here. In Colab you can upload, then use /content/filename.jpg
primary_path = "/content/data/tiny-imagenet-200/train/n03100240/images/n03100240_103.JPEG"
secondary_path = "/content/data/tiny-imagenet-200/train/n01443537/images/n01443537_10.JPEG"

make_augmentation_row(
    primary_path,
    secondary_path,
    primary_label="Convertible",
    secondary_label="Goldfish",
    size=(64, 64),          # match Tiny ImageNet; use (32,32) for CIFAR
    mixup_lam=0.5,
    cutout_frac=0.25,
    cutout_fill=0.0,
    cutout_center=(16, 40),     # or (32,32) for deterministic center
    cutmix_lam=0.75,
    cutmix_bbox=(24, 0, 56, 32),       # or set fixed box like (10,10,40,40)
    seed=3,
    out_path="aug_row_tinyimagenet.pdf",
    dpi=500,
)
